In [1]:
import os
os.environ["PYOPENGL_PLATFORM"] = "egl" #opengl seems to only work with TPU
import numpy as np
import cv2
import torch
import glob
import trimesh
import pyrender
from pyrender import IntrinsicsCamera,Mesh, Node, Scene,OffscreenRenderer
from utils.io import load_root
from nnutils.geom_utils import obj_to_cam

In [2]:
img_type='vert'
scale = 0.5
dep_scale = 0.2
checkpoint_dir = "logdir"
#seqname = "catamelie-dualrig002-properlyscaledcams-leftcam-ds-load-fgbkgd"
#seqname = "human-dualrig002-properlyscaledcams-leftcam-ds-load-fgbkgd"
seqname = "dog-dualrig001-leftcam-finetune-fgbkgd"
#seqname = "catamelie-dualrig003-depwt5-entwt0p01-freezebkgdshape-leftcam-finetune-fgbgkd"

obj_dirs = sorted(glob.glob("{}/{}/obj[0-9]".format(checkpoint_dir, seqname)))
loadname_objs = ["{}/{}/obj{}/".format(checkpoint_dir, seqname, obj_index) for obj_index in range(len(obj_dirs))]
rootdir_objs = ["{}/{}/obj{}/".format(checkpoint_dir, seqname, obj_index) for obj_index in range(len(obj_dirs))]

In [3]:
# loading optimized cameras poses
rtks_objs = []
for rootdir in rootdir_objs:
    rtks = load_root(rootdir, 0)  # cap frame=0=>load all
    rtks = rtks.astype(np.float32)
    rtks[:,3] = rtks[:,3] * scale
    rtks_objs.append(rtks)

rtks_objs_torch = [torch.Tensor(rtks_obj).cuda() for rtks_obj in rtks_objs]
#bkgd2cam = np.concatenate((rtks_bkgd[:, :3, :], np.repeat(np.array([[[0., 0., 0., 1.]]]), rtks_bkgd.shape[0], axis = 0)), axis = 1)
sample_idx = np.arange(0, len(rtks_objs[-1])-1).astype(int)

In [4]:
# setting params b4 loading meshes
size = len(rtks_objs[-1])-1
rndsils = np.ones((size, 960, 720))             # (N, H, W)        # set opts.scale to 0.375
img_size = rndsils[0].shape

if img_size[0] > img_size[1]:
    img_type='vert'
else:
    img_type='hori'

img_size = int(max(img_size)*scale)

if img_type=='vert':
    size_short_edge = int(rndsils[0].shape[1] * img_size/rndsils[0].shape[0])
    height = img_size
    width = size_short_edge        
else:
    size_short_edge = int(rndsils[0].shape[0] * img_size/rndsils[0].shape[1])
    height = size_short_edge
    width = img_size

direc_l = pyrender.DirectionalLight(color=np.ones(3), intensity=6.0)

In [5]:
# loading bkgd meshes and compute near-far plane for every frame

near_far_values = []

for i, frame_idx in enumerate(sample_idx):
    print("frame number : {}".format(frame_idx))
    r = OffscreenRenderer(img_size, img_size)
    scene = Scene(ambient_light=0.4*np.asarray([1.,1.,1.,1.]))
    
    for obj_index, rootdir in enumerate(rootdir_objs):
        meshdir_obj = rootdir                                  # "logdir/{}/obj0/".format(opts.seqname)
        rtks_obj = torch.Tensor(rtks_objs[obj_index]).cuda()
    
        # 1. load meshes
        meshdir_obj_time = glob.glob(meshdir_obj + "*-mesh-%05d.obj"%(frame_idx))
        assert(len(meshdir_obj_time) == 1)
        mesh_obj_time = trimesh.load(meshdir_obj_time[0], process=False)

        mesh_obj_time.visual.vertex_colors[:, :3] = 64
        if obj_index < len(rootdir_objs) - 1:
            mesh_obj_time.visual.vertex_colors[:, obj_index] = 160     # obj0 (foreground): red, # obj 1 (background): green

        # 2. using rtks_objs, which represent obj2cam transformations, convert mesh vertices coords into the camera-space
        faces_obj_time = torch.Tensor(mesh_obj_time.faces[None]).cuda()
        verts_obj_time = torch.Tensor(mesh_obj_time.vertices[None]).cuda()
        Rmat_obj_time = rtks_objs_torch[obj_index][i : i+1, :3, :3]
        Tmat_obj_time = rtks_objs_torch[obj_index][i : i+1, :3, 3]

        verts_obj_time = obj_to_cam(verts_obj_time, Rmat_obj_time, Tmat_obj_time)           # need to input Rmat of shape (1, 3, 3) and Tmat of shape (1,3), where Rmat, Tmat denote obj2cam matrices
        mesh_obj_time = trimesh.Trimesh(vertices=np.asarray(verts_obj_time[0,:,:3].cpu()), faces=np.asarray(faces_obj_time[0].cpu()), vertex_colors=mesh_obj_time.visual.vertex_colors)
        meshr_obj_time = Mesh.from_trimesh(mesh_obj_time, smooth=True)
        meshr_obj_time._primitives[0].material.RoughnessFactor=1.

        # 3. add the resulting mesh, where vertices are no defined in the desired camera frame to a Scene object
        scene.add_node(Node(mesh=meshr_obj_time))

    # 4. add a camera node with an I_4x4 transformation (adjusted for sign conventions)
    focal_time = rtks_objs_torch[-1][i, 3, :2]
    ppoint_time = rtks_objs_torch[-1][i, 3, 2:]
    cam_time = IntrinsicsCamera(
        focal_time[0],
        focal_time[1],
        ppoint_time[0],
        ppoint_time[1],
        znear=1e-3,zfar=1000)
    cam_pose = -np.eye(4); cam_pose[0,0]=1; cam_pose[-1,-1]=1
    cam_node = scene.add(cam_time, pose=cam_pose)

    # 5. add a light source to the scene
    theta = 9*np.pi/9
    init_light_pose = np.asarray([[1,0,0,0],[0,np.cos(theta),-np.sin(theta),0],[0,np.sin(theta),np.cos(theta),0],[0,0,0,1]])
    light_pose = init_light_pose
    direc_l_node = scene.add(direc_l, pose=light_pose)

    # 6. render Scene
    mesh_rnd_color_time, mesh_rnd_depth_time = r.render(scene,flags=pyrender.RenderFlags.SHADOWS_DIRECTIONAL | pyrender.RenderFlags.SKIP_CULL_FACES)
    r.delete()
    mesh_rnd_depth_time = mesh_rnd_depth_time[:height, :width]

    #cv2.imwrite('%smeshcolor_%05d.png'%(rootdir_bkgd,i), mesh_rnd_color_time[...,::-1])
    
    near = np.min(mesh_rnd_depth_time[mesh_rnd_depth_time > 0])
    far = np.max(mesh_rnd_depth_time[mesh_rnd_depth_time > 0])
    near_far = np.stack([near, far])
    print("near - far (m): {}, {}".format(near / dep_scale, far / dep_scale))
    near_far_values.append(near_far)

# scale back to metric space (i.e. in meters)
near_far_values = np.stack(near_far_values, axis = 0) / dep_scale                       # (size, 2)
near_far_overallframes = [np.min(near_far_values[:, 0]), np.max(near_far_values[:, 1])]
near_far_overallframes = np.stack(near_far_overallframes)
print("near_far_overallframes: {}".format(near_far_overallframes))

np.save(os.path.join(checkpoint_dir, seqname, "nvs-inputview-nf_perframe.npy"), near_far_values)
np.save(os.path.join(checkpoint_dir, seqname, "nvs-inputview-nf_overallframes.npy"), near_far_overallframes)

frame number : 0
near - far (m): 0.9208633005619049, 5.223490595817566
frame number : 1
near - far (m): 0.9462698549032211, 5.51761269569397
frame number : 2
near - far (m): 0.95644511282444, 5.5748748779296875
frame number : 3
near - far (m): 0.9610003978013992, 5.649849772453308
frame number : 4
near - far (m): 0.9213552623987198, 5.763331651687622
frame number : 5
near - far (m): 0.9206977486610413, 5.852055549621582
frame number : 6
near - far (m): 0.9152254462242126, 5.914374589920044
frame number : 7
near - far (m): 0.9393579512834549, 5.84892213344574
frame number : 8
near - far (m): 0.9881964325904846, 5.797593593597412
frame number : 9
near - far (m): 0.9694215655326843, 5.8537280559539795
frame number : 10
near - far (m): 0.9714275598526001, 5.897343754768372
frame number : 11
near - far (m): 1.0279963165521622, 5.858123302459717
frame number : 12
near - far (m): 1.0892964899539948, 5.97345769405365
frame number : 13
near - far (m): 1.044604852795601, 6.08731210231781
frame n